# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst, as this task requires statistical modeling and machine learning expertise.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "Data Analyst",
    "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning",
    "context": "The task involves analyzing market data for AAPL to identify trends and predict market movements. Use statistical modeling and machine learning techniques to gain insights and identify significant market opportunities or threats."
}


> Entering new CrewAgentExecutor chain...
I need to gather the most up-to-date market

Final Answer: 
Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
Oops, something went wrong Skip to navigation Skip to main content Skip to right column News Today's news US Politics World Weather Climate change Health Wellness Mental health Sexual health Dermatology Oral health Hair loss Foot health Nutrition Healthy eating Meal delivery Weight loss Vitamins and supplements Fitness Equipment Exercise Women’s health Sleep Healthy aging Hearing Mobility Science Originals The 360 Newsletters Games Life Health Wellness Nutrition Fitness Healthy aging Mental health Sleep Your body Dermatology Children's health Foot health Hair loss Hearing Oral health Sexual health Women’s health Conditions Cardiovascular health Digestive health Endocrine system Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Style Accessories Clothing Luggage Shoes Beauty Hair Makeup Skincare Sunscreen Health Dental Fitness Hair loss Hearing aids Men

Final Answer: 
Apple Inc. (AAPL) is a leading technology company that designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers a wide range of products and services, including the iPhone, Mac, iPad, Apple Watch, and Apple TV. Additionally, Apple provides subscription-based services such as Apple Arcade, Apple Fitness+, Apple Music, Apple News+, and Apple TV+.

As of November 11, 2025, AAPL's stock price closed at $275.73, reflecting a 0.17% increase. The company's market cap stands at $4.067 trillion, with a P/E ratio of 36.90. AAPL has a strong financial position, with a profit margin of 26.92% and a return on equity of 171.42%.

AAPL's performance has been positive, with a YTD return of 10.41% and a 1-year return of 23.31%. Analysts have set a price target range of $215.00 to $345.00 for AAPL, with an average target of $280.46. The company's valuation metrics and financial highlights indicate a solid positio

Final Answer: Risk tolerance is the degree of uncertainty and financial loss that an investor is willing to take on an investment in exchange for a possible higher return. It is an important component in investing because it often determines what kind of investments you choose. Investors with a higher risk tolerance often invest in highly volatile assets like stocks and equity funds while those with a low risk tolerance may choose bonds and other fixed-income assets. Your age, investment goals, and income can contribute to your risk tolerance. Factors like your age, investment goals, and income contribute to your risk tolerance. Aggressive investors are willing to risk losing money to potentially get better results, while moderate investors want to grow their money without losing too much. Conservative investors are willing to accept little to no volatility in their investment portfolios. Assessing an investor's risk tolerance is crucial in determining their investment strategies and p

Final Answer: 
Title: Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
Link: https://finance.yahoo.com/quote/AAPL/
Snippet: Apple's cautious AI spending strategy has led to a 31% stock rally in the latter half of 2025, distinguishing it from competitors amid a ...

Title: Apple Inc. Stock Quote (U.S.: Nasdaq) - AAPL - MarketWatch
Link: https://www.marketwatch.com/investing/stock/aapl?gaa_at=eafs&gaa_n=AWEtsqd9hlmsf57-5UKrYcnBE6WVMkRXikKdqv7Ix9Vm1C042wbKrvPqvHbN&gaa_ts=6913faab&gaa_sig=yYzpId0Eo6r99lpLoqrNntNFaposh_cic8EJz3R19z1vQJFyfb7FL_Ws5649OV9ekK3h-kcHXUBB5DvOMajLxw%3D%3D
Snippet: Apple Inc. ; Open $269.81 ; Day Range 269.80 - 275.91 ; 52 Week Range 169.21 - 277.32 ; Market Cap $3.98T ; Public Float 14.76B ...

Title: AAPL: Apple Inc - Stock Price, Quote and News - CNBC
Link: https://www.cnbc.com/quotes/AAPL
Snippet: Apple Inc AAPL:NASDAQ ; Close. 275.25 quote price arrow up +5.82 (+2.16%) ; Volume. 44,585,939 ; 52 week range. 169.21 - 277.32.

> Finished chain.

I need to delegate the task of evaluating the risks associated with the proposed trading strategies and execution plans for AAPL to the Risk Advisor. They have the expertise to assess potential risks and recommend mitigation strategies.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "Risk Advisor",
    "task": "Evaluate the risks associated with the proposed trading strategies and execution plans for AAPL",
    "context": "Detailed execution plans for the approved trading strategies for AAPL should be developed taking into account the insights on the current market conditions for AAPL and the specific trading strategies for day trading, swing trading, and long-term investment. By utilizing a combination of these strategies based on the market conditions, we can maximize profits and minimize risks. It is crucial to carefully analyze the information gathered from MarketWatch and other sources to tailor the execution plans to fit the specific goals and risk tolerance 

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Dear Team,

Please find below the comprehensive risk analysis report for the proposed trading strategies and execution plans for AAPL:

Risk Analysis Report for AAPL:

1. Market Risk:
- The current market conditions for AAPL are volatile due to external factors such as global economic uncertainty and geopolitical tensions. This poses a risk to the trading strategies as sudden market fluctuations can impact the profitability of the investments.
- Mitigation Strategy: Diversification of trading strategies to include a mix of day trading, swing trading, and long-term investment to spread the risk across different market conditions.

2. Operational Risk:
- The execution plans for AAPL may face operational risks such as technology failures, errors in trade execution, or regulatory changes that could disrupt the trading process.
- Mitigation Strategy: Implementing robust operational procedures and monitoring systems to minimize the impact of operational risks on the trading strategies.

3. Liquidity Risk:
- The liquidity of AAPL stocks may vary depending on market demand and supply, which can affect the ability to buy or sell shares at desired prices.
- Mitigation Strategy: Maintaining adequate liquidity reserves and closely monitoring market liquidity to ensure smooth execution of trading strategies.

4. Credit Risk:
- Counterparty credit risk is a concern when engaging in trading activities, as default by a trading partner can lead to financial losses.
- Mitigation Strategy: Conducting thorough credit assessments of counterparties and setting limits on exposure to minimize credit risk.

5. Regulatory Risk:
- Changes in regulatory requirements or enforcement actions by regulatory authorities could impact the trading strategies for AAPL.
- Mitigation Strategy: Staying informed about regulatory developments and ensuring compliance with all applicable laws and regulations to mitigate regulatory risks.

Overall, it is essential to continuously monitor and assess the risks associated with the trading strategies for AAPL and implement proactive measures to mitigate potential risks. By following the recommended mitigation strategies, we can optimize the profitability of the investments while minimizing the exposure to risks.

Thank you for your attention to this report.

Best regards,
Risk Advisor